In [1]:
from prj.agents.factory import AgentsFactory
import os
path = "/home/lorecampa/projects/jane_street_forecasting/experiments/train/lgbm/lgbm_1seeds_850_1529-0.2_20241224_183602_20241225_121900/train/model"
agent_dict = {'agent_type': 'lgbm', 'load_path': path}

agent = AgentsFactory.load_agent(agent_dict)
agent

2024-12-25 14:56:34.066411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 14:56:34.066467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 14:56:34.067767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 14:56:34.077530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-25 14:56:35.354664: W tensorflow/compiler/tf2

Loading models, overwriting seeds: [293461583]


In [2]:
from prj.config import DATA_DIR
from prj.data.data_loader import DataConfig, DataLoader

data_args = {'include_intrastock_norm': True}
config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)

In [3]:
X, y, w, _ = loader.load_numpy_with_partition(start_part_id=9, end_part_id=9)
X.shape, y.shape, w.shape

((6274576, 133), (6274576,), (6274576,))

In [4]:
from tqdm import tqdm
import numpy as np

def predict_with_progress(agent, X, chunk_size=100000):
    predictions = []
    for i in tqdm(range(0, len(X), chunk_size)):
        predictions.extend(agent.predict(X[i:i+chunk_size]))
    return np.array(predictions)

y_pred = predict_with_progress(agent, X, chunk_size=200000)
y_pred.shape

100%|██████████| 32/32 [03:02<00:00,  5.71s/it]


(6274576,)

In [5]:
from prj.metrics import weighted_mae, weighted_mse, weighted_r2, weighted_rmse

def metrics(y_true, y_pred, weights=None):
    return {
        'r2_w': weighted_r2(y_true, y_pred, weights=weights),
        'mae_w': weighted_mae(y_true, y_pred, weights=weights),
        'mse_w': weighted_mse(y_true, y_pred, weights=weights),
        'rmse_w': weighted_rmse(y_true, y_pred, weights=weights),
    }

metrics(y, y_pred, weights=w)

{'r2_w': 0.009828263309537455,
 'mae_w': 0.5193012293405588,
 'mse_w': 0.627753315181394,
 'rmse_w': 0.7923088508791215}

In [6]:
# {'r2_w': 0.010407868259883046,
#  'mae_w': 0.519212864150282,
#  'mse_w': 0.6273858547544875,
#  'rmse_w': 0.7920769247708757}